In [1]:
import pickle as pk
import numpy as np
import pandas as pd
import re

In [2]:
sampleDf = pd.read_pickle('D://TDE/CharEncoder-WordDecoderS2S/data/final_train14_09_2017.pkl')

In [3]:
# ['<acc_num>', '<account_type>', '<ads>', '<amount>', '<atm_id>',
#        '<bank_name>', '<card_num>', '<cash_back>', '<check_num>', '<city>',
#        '<client_name>', '<constant>', '<country>', '<curr>', '<email>',
#        '<intmd>', '<itin>', '<mcc_code>', '<other>', '<person_name>',
#        '<phone_number>', '<policy_num>', '<pos_id>', '<pymt_type>',
#        '<pymt_vendor>', '<pyrl_id>', '<sic>', '<ssn>', '<state>', '<store_id>',
#        '<street>', '<trans_date>', '<trans_num>', '<trans_purp>',
#        '<unidentified>', '<vendor_name>', '<zip>', 'cobrand_id',
#        'description_unmasked', 'fileName', 'index', 'ledger_entry',
#        'merchant_name', 'res_category', 'sum_info_id', 'tokenlist',
#        'trans_id_type', 'type', 'unique_transaction_id', 'wordlist', 'filter',
#        'inTop_merchants', 'one_occurence_merchants', 'isMerchant',
#        'inTop_suminfos', 'one_occurence_suminfos', 'in_test',
#        'isUnseenMerchant', 'isUnseenSuminfo']

In [4]:
sampleDf = sampleDf[['description_unmasked', 'ledger_entry', 'merchant_name', 'tokenlist', 'type', 'unique_transaction_id']]
sampleDf = sampleDf.dropna(how= 'any', subset = ['merchant_name'])
sampleDf['merchant_name'] = sampleDf['merchant_name'].str.upper()
needed_merchs = sampleDf['merchant_name'].str.upper().unique()

In [5]:
mapping_merch_list = pd.read_csv('D:/TDE/Peyush_MerClassification/CNNData/yodlee_factual_filled_2.6v.csv', sep = '|')
mapping_merch_list = mapping_merch_list[['yodlee_name','ht_name', 'factual_name', 'tde_name', 'quant_name','dc_name']]
mapping_merch_list = mapping_merch_list.apply(lambda x: x.astype(str).str.upper())
mapping_merch_list = mapping_merch_list.drop_duplicates()
mapping_merch_list = mapping_merch_list.dropna()

#Select those QUANT merchant names that are in tde list
refined_mapping_merch_list_1 = mapping_merch_list.loc[mapping_merch_list['tde_name'].isin(needed_merchs)]

#Select those QUANT merchant names that are in DC list
refined_mapping_merch_list_2 = mapping_merch_list.loc[mapping_merch_list['dc_name'].isin(needed_merchs)]

#Select those QUANT merchant names that are in HTlist list
refined_mapping_merch_list_3 = mapping_merch_list.loc[mapping_merch_list['ht_name'].isin(needed_merchs)]

#Select those QUANT merchant names that are in quant list
refined_mapping_merch_list_4 = mapping_merch_list.loc[mapping_merch_list['quant_name'].isin(needed_merchs)]

#Select those QUANT merchant names that are in factual_name list
refined_mapping_merch_list_5 = mapping_merch_list.loc[mapping_merch_list['factual_name'].isin(needed_merchs)]

#Select those YODLEE_NAME merchant names that are in factual_name list
refined_mapping_merch_list_6 = mapping_merch_list.loc[mapping_merch_list['yodlee_name'].isin(needed_merchs)]

refined_mapping_merch_list = refined_mapping_merch_list_1.append([refined_mapping_merch_list_2, refined_mapping_merch_list_3,
                                                                 refined_mapping_merch_list_4, refined_mapping_merch_list_5,
                                                                 refined_mapping_merch_list_6]\
                                                                 , ignore_index=True).drop_duplicates()
refined_mapping_merch_list = refined_mapping_merch_list.dropna()

print (refined_mapping_merch_list.shape)



(165286, 6)


In [6]:
#Combine the above 6 lists, each column mapped from any_name -> yodlee_name
refined_mapping_merch_list_1 = refined_mapping_merch_list_1[['tde_name', 'yodlee_name']].rename(columns = {'tde_name':'any_name'})

refined_mapping_merch_list_2 = refined_mapping_merch_list_2[['dc_name', 'yodlee_name']].rename(columns = {'dc_name':'any_name'})

refined_mapping_merch_list_3 = refined_mapping_merch_list_3[['ht_name', 'yodlee_name']].rename(columns = {'ht_name':'any_name'})

refined_mapping_merch_list_4 = refined_mapping_merch_list_4[['quant_name', 'yodlee_name']].rename(columns = {'quant_name':'any_name'})

refined_mapping_merch_list_5 = refined_mapping_merch_list_5[['factual_name', 'yodlee_name']].rename(columns = {'factual_name':'any_name'})


comb_refined_mapping_merch = pd.concat([refined_mapping_merch_list_1, refined_mapping_merch_list_2, refined_mapping_merch_list_3,\
                                       refined_mapping_merch_list_4, refined_mapping_merch_list_5], ignore_index=True)
comb_refined_mapping_merch = comb_refined_mapping_merch.rename(columns = {'any_name': 'merchant_name'})
comb_refined_mapping_merch = comb_refined_mapping_merch.drop_duplicates()
comb_refined_mapping_merch = comb_refined_mapping_merch.drop_duplicates(subset = ['merchant_name'])
print (comb_refined_mapping_merch.shape)


(165261, 2)


In [7]:
# Taking joins with the said coulmn and fill yodlee_name
sampleDf = sampleDf.merge(comb_refined_mapping_merch, how='left', on = ['merchant_name'])
sampleDf['yodlee_name'].fillna('OTHER', inplace=True)

In [8]:
# This needs to be used during traiing data creation
tgt_dict = {i+1: v for (i, v) in enumerate(sampleDf['yodlee_name'].unique())} # Incresing key value
tgt_dict[len(tgt_dict.keys()) + 1] = '<u>'
print (len(tgt_dict.keys()))
tgt_rev_dict = {v: k for k, v in tgt_dict.items()}

import json
with open('../model_files/tgtDict.txt', 'w') as fp:
    json.dump(tgt_dict, fp)

164252


In [9]:
#Shuffle data
sampleDf = sampleDf.sample(frac=1)

In [10]:
# Save source file and target file
import sys

sys.path.append('..')
from utils.tokenization import basic_tokenizer

sampleDf['tokenized_desc'] = sampleDf.apply(lambda x: basic_tokenizer(x['description_unmasked'], convert_lower=True), axis = 1)

In [11]:
sampleDf = sampleDf.sample(frac=1)

In [55]:
# Remove transaction with unequal desc length
lenUnequal_inx = (sampleDf.apply(lambda x: len(x['tokenized_desc']) == len(x['tokenlist']), axis = 1)).values
sampleDf = sampleDf[lenUnequal_inx]

In [56]:
def change_token(token_list, merchant_name):
    
    last_entry_index = [i for i, j in enumerate(token_list) if j == '<vendor_name>']
   
    if not last_entry_index:
        last_entry_index = -1
        token_list = ['<u>' for i,x in enumerate(token_list)]
    else:
        last_entry_index = last_entry_index[-1]
        token_list[last_entry_index] = merchant_name
        if(merchant_name =='OTHER'):
            token_list[last_entry_index] = '<u>'
        token_list = ['<u>' if x != merchant_name else x for i,x in enumerate(token_list)]
    token_list = [tgt_rev_dict[t] for t in token_list]
    return token_list
    
sampleDf['mod_tokenlist'] = sampleDf.apply(lambda x: change_token(x['tokenlist'], x['yodlee_name']), axis = 1)

c:\users\psopori\appdata\local\continuum\anaconda3\envs\vtf\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [57]:
# Entries where length does not match..check if need to be removed
sampleDf[sampleDf['mod_tokenlist'].str.len() != sampleDf['mod_tokenlist'].str.len()][['tokenized_desc', 'mod_tokenlist']]

,tokenized_desc,mod_tokenlist


In [58]:
# sampleDf['tokenized_desc'].apply(lambda x: str(' '.join(x)).strip('"')).to_csv('srcData.txt', header=None, index=None, sep=' ', mode='w')
# sampleDf['mod_tokenlist'].apply(lambda x: (' '.join([str(v) for v in x])).strip('"')).to_csv('tgtData.txt', header=None, index=None, sep=' ', mode='w')

In [59]:
#tl = sampleDf[sampleDf['yodlee_name'] == 'WELLS FARGO & CO.']['tokenlist'].values
#tl

In [60]:
# np.savetxt('srcData.txt', (sampleDf['tokenized_desc'].apply(lambda x: (' '.join(x)).strip('"'))).values, fmt='%s', delimiter='\n')
# np.savetxt('tgtData.txt', sampleDf['mod_tokenlist'].apply(lambda x: (' '.join([str(v) for v in x])).strip('"')), fmt='%s', delimiter='\n')

In [61]:
with open('D://TDE/CharEncoder-WordDecoderS2S/data/srcData.txt', mode='wt', encoding='utf-8') as myfile:
    for lines in sampleDf['tokenized_desc'].apply(lambda x: (' '.join(x)).strip('"')).values:
        myfile.write(lines)
        myfile.write('\n')
        
with open('D://TDE/CharEncoder-WordDecoderS2S/data/tgtData.txt', mode='wt', encoding='utf-8') as myfile:
    for lines in sampleDf['mod_tokenlist'].apply(lambda x: (' '.join([str(v) for v in x])).strip('"')).values:
        myfile.write(lines)
        myfile.write('\n')



In [64]:
np.unique((sampleDf.apply(lambda x: len(x['tokenized_desc']) == len(x['mod_tokenlist']), axis = 1)).values)

array([ True], dtype=bool)

# Test data creation

In [66]:
import pandas as pd
testDf = pd.read_pickle('D://TDE/CharEncoder-WordDecoderS2S/data/final_test14_09_2017.pkl')

In [67]:
testDf = testDf[['description_unmasked', 'ledger_entry', 'merchant_name', 'tokenlist', 'type', 'unique_transaction_id']]
testDf = testDf.dropna(how= 'any', subset = ['merchant_name'])
testDf['merchant_name'] = testDf['merchant_name'].str.upper()
needed_merchs = testDf['merchant_name'].str.upper().unique()

In [68]:
mapping_merch_list = pd.read_csv('D:/TDE/Peyush_MerClassification/CNNData/yodlee_factual_filled_2.6v.csv', sep = '|')
mapping_merch_list = mapping_merch_list[['yodlee_name','ht_name', 'factual_name', 'tde_name', 'quant_name','dc_name']]
mapping_merch_list = mapping_merch_list.apply(lambda x: x.astype(str).str.upper())
mapping_merch_list = mapping_merch_list.drop_duplicates()
mapping_merch_list = mapping_merch_list.dropna()

#Select those QUANT merchant names that are in tde list
refined_mapping_merch_list_1 = mapping_merch_list.loc[mapping_merch_list['tde_name'].isin(needed_merchs)]

#Select those QUANT merchant names that are in DC list
refined_mapping_merch_list_2 = mapping_merch_list.loc[mapping_merch_list['dc_name'].isin(needed_merchs)]

#Select those QUANT merchant names that are in HTlist list
refined_mapping_merch_list_3 = mapping_merch_list.loc[mapping_merch_list['ht_name'].isin(needed_merchs)]

#Select those QUANT merchant names that are in quant list
refined_mapping_merch_list_4 = mapping_merch_list.loc[mapping_merch_list['quant_name'].isin(needed_merchs)]

#Select those QUANT merchant names that are in factual_name list
refined_mapping_merch_list_5 = mapping_merch_list.loc[mapping_merch_list['factual_name'].isin(needed_merchs)]

#Select those YODLEE_NAME merchant names that are in factual_name list
refined_mapping_merch_list_6 = mapping_merch_list.loc[mapping_merch_list['yodlee_name'].isin(needed_merchs)]

refined_mapping_merch_list = refined_mapping_merch_list_1.append([refined_mapping_merch_list_2, refined_mapping_merch_list_3,
                                                                 refined_mapping_merch_list_4, refined_mapping_merch_list_5,
                                                                 refined_mapping_merch_list_6]\
                                                                 , ignore_index=True).drop_duplicates()
refined_mapping_merch_list = refined_mapping_merch_list.dropna()

print (refined_mapping_merch_list.shape)


(19554, 6)


In [69]:
#Combine the above 6 lists, each column mapped from any_name -> yodlee_name
refined_mapping_merch_list_1 = refined_mapping_merch_list_1[['tde_name', 'yodlee_name']].rename(columns = {'tde_name':'any_name'})

refined_mapping_merch_list_2 = refined_mapping_merch_list_2[['dc_name', 'yodlee_name']].rename(columns = {'dc_name':'any_name'})

refined_mapping_merch_list_3 = refined_mapping_merch_list_3[['ht_name', 'yodlee_name']].rename(columns = {'ht_name':'any_name'})

refined_mapping_merch_list_4 = refined_mapping_merch_list_4[['quant_name', 'yodlee_name']].rename(columns = {'quant_name':'any_name'})

refined_mapping_merch_list_5 = refined_mapping_merch_list_5[['factual_name', 'yodlee_name']].rename(columns = {'factual_name':'any_name'})


comb_refined_mapping_merch = pd.concat([refined_mapping_merch_list_1, refined_mapping_merch_list_2, refined_mapping_merch_list_3,\
                                       refined_mapping_merch_list_4, refined_mapping_merch_list_5], ignore_index=True)
comb_refined_mapping_merch = comb_refined_mapping_merch.rename(columns = {'any_name': 'merchant_name'})
comb_refined_mapping_merch = comb_refined_mapping_merch.drop_duplicates()
comb_refined_mapping_merch = comb_refined_mapping_merch.drop_duplicates(subset = ['merchant_name'])
print (comb_refined_mapping_merch.shape)


(19312, 2)


In [70]:
# Taking joins with the said coulmn and fill yodlee_name
testDf = testDf.merge(comb_refined_mapping_merch, how='left', on = ['merchant_name'])
testDf['yodlee_name'].fillna('OTHER', inplace=True)

In [73]:
#Shuffle data
testDf = testDf.sample(frac=1)

In [74]:
# Save source file and target file
import sys

sys.path.append('..')
from utils.tokenization import basic_tokenizer

testDf['tokenized_desc'] = testDf.apply(lambda x: basic_tokenizer(x['description_unmasked'], convert_lower=True), axis = 1)

In [75]:
# Remove transaction with unequal desc length
lenUnequal_inx = (testDf.apply(lambda x: len(x['tokenized_desc']) == len(x['tokenlist']), axis = 1)).values
testDf = testDf[lenUnequal_inx]

In [76]:
testDf = testDf.sample(frac=1)

In [77]:
# This needs to be used during traiing data creation

import json
with open('../model_files/tgtDict.txt', 'r') as fp:
    tgt_dict = json.load(fp)
    
print (len(tgt_dict.keys()))
tgt_rev_dict = {v: k for k, v in tgt_dict.items()}

164252


In [78]:
tgt_rev_dict

{'OTHER': '1',
 'PAYPAL': '2',
 'THE FACE SHOP': '3',
 'MIDAS': '4',
 'NEW YORK & COMPANY': '5',
 'NATIONAL CAR RENTAL': '6',
 'OLD NAVY': '7',
 'GOVERNMENT': '8',
 'PARK AVENUE THRIFT': '9',
 'PAPA MURPHYS TAKE N BAKE': '10',
 'PARK AVENUE PROPERTIES': '11',
 'NORTHWEST FEDERAL CREDIT UNION': '12',
 'EBAY': '13',
 'ORION FEDERAL CREDIT UNION': '14',
 'THE PINK LILY BOUTIQUE': '15',
 'ABC FINE WINES & SPIRITS': '16',
 'ABC PHARMACY': '17',
 'ACTBLUE': '18',
 'ADAFRUIT INDUSTRIES': '19',
 'ADOBE SYSTEMS': '20',
 'ADORAMA': '21',
 'AGAPE ANIMAL CENTER': '22',
 'SUBMARINA': '23',
 'DANDELION CHOCOLATE': '24',
 'THE LITTLE TART BAKESHOP': '25',
 'DOLLAR TREE': '26',
 'HOPE THRIFT': '27',
 'THE HYPPO': '28',
 'CAMPUS COFFEE BEAN': '29',
 'OAK BARREL TAVERN': '30',
 "PARSON'S CHICKEN & FISH": '31',
 'BP GLOBAL': '32',
 'SORELLE': '33',
 'CRAYOLA EXPERIENCE': '34',
 'THE END ZONE': '35',
 'THE FEVE': '36',
 'THE FLORIDA ORCHESTRA': '37',
 'SPIRIT HALLOWEEN': '38',
 "MARINERS' MUSEUM": '39',
 

In [79]:
testDf['yodlee_name'] = testDf.apply((lambda x: x['yodlee_name'] if x['yodlee_name'] in tgt_rev_dict else 'OTHER'), axis = 1)

In [80]:
def change_token(token_list, merchant_name):
    
    last_entry_index = [i for i, j in enumerate(token_list) if j == '<vendor_name>']
   
    if not last_entry_index:
        last_entry_index = -1
        token_list = ['<u>' for i,x in enumerate(token_list)]
    else:
        last_entry_index = last_entry_index[-1]
        token_list[last_entry_index] = merchant_name
        if(merchant_name =='OTHER'):
            token_list[last_entry_index] = '<u>'
        token_list = ['<u>' if x != merchant_name else x for i,x in enumerate(token_list)]
    token_list = [tgt_rev_dict[t] for t in token_list]
    return token_list
    
testDf['mod_tokenlist'] = testDf.apply(lambda x: change_token(x['tokenlist'], x['yodlee_name']), axis = 1)

In [81]:
# Entries where length does not match..check if need to be removed
testDf[testDf['mod_tokenlist'].str.len() != testDf['mod_tokenlist'].str.len()][['tokenized_desc', 'mod_tokenlist']]

,tokenized_desc,mod_tokenlist


In [82]:
testDf.shape

(155275, 9)

In [83]:
stratify_col = 'yodlee_name'
test_size= 0.5
random_state=42
df = testDf
from sklearn.model_selection import train_test_split
# duplicate trans of strafied col values whose count is 1 so that startification logic will work
col_val_cnts = df[stratify_col].value_counts()
cnt_1_val = col_val_cnts[col_val_cnts==1]

for cat in cnt_1_val.index.values:
    df_cat_1_cnt = df[df[stratify_col] == cat]
    df = df.append([df_cat_1_cnt],ignore_index=True)

testDf_,valDf_ = train_test_split(df,test_size=test_size,stratify= df[stratify_col],random_state=random_state)

In [84]:
print (testDf_.shape, valDf_.shape)

(81586, 9) (81586, 9)


In [85]:
# Save test data
with open('D://TDE/CharEncoder-WordDecoderS2S/data/srcTestData.txt', mode='wt', encoding='utf-8') as myfile:
    for lines in testDf_['tokenized_desc'].apply(lambda x: (' '.join(x)).strip('"')).values:
        myfile.write(lines)
        myfile.write('\n')
        
with open('D://TDE/CharEncoder-WordDecoderS2S/data/tgtTestData.txt', mode='wt', encoding='utf-8') as myfile:
    for lines in testDf_['mod_tokenlist'].apply(lambda x: (' '.join([str(v) for v in x])).strip('"')).values:
        myfile.write(lines)
        myfile.write('\n')
        
# Save val data
with open('D://TDE/CharEncoder-WordDecoderS2S/data/srcDevData.txt', mode='wt', encoding='utf-8') as myfile:
    for lines in valDf_['tokenized_desc'].apply(lambda x: (' '.join(x)).strip('"')).values:
        myfile.write(lines)
        myfile.write('\n')
        
with open('D://TDE/CharEncoder-WordDecoderS2S/data/tgtDevData.txt', mode='wt', encoding='utf-8') as myfile:
    for lines in valDf_['mod_tokenlist'].apply(lambda x: (' '.join([str(v) for v in x])).strip('"')).values:
        myfile.write(lines)
        myfile.write('\n')


# TEst RUN OF SAMPLE

In [ ]:


from tensorflow.contrib import learn
import sys
import numpy as np
sys.path.append('..')
from utils.tokenization import basic_tokenizer
vocab_processor = learn.preprocessing.VocabularyProcessor(22)
with open("srcData.txt") as f:
    content = f.readlines()
content = [c.strip().strip('"') for c in content]
vocab_processor = vocab_processor.fit(content, unused_y=None)
#vocab_dict = vocab_processor.fit(content, unused_y=None)
#vocab_dict.save('a.dict')

In [ ]:
arr = np.array(list(vocab_processor.fit_transform((content))))

In [ ]:
list([6])[-1]

In [2]:
import tensorflow as tf

In [3]:
import os
import sys

sys.path.append('..')
from utils import misc_utils as utils
from utils import vocab_utils
from utils import iterator_utils
from utils import evaluation_utils
from tensorflow.python.ops import lookup_ops
from parameters import params
src_vocab_file = '../model_files/srcVocab.vocab'#Vocab file for src, created from data file if file doesnt exist, path should be correct
tgt_vocab_file = '../model_files/tgtVocab.vocab'#Vocab file for tgt, created from data file
enc_char_map_path  = "../model_files/char_vocab.vocab"

src_vocab_table, tgt_vocab_table = vocab_utils.create_vocab_tables(src_vocab_file, tgt_vocab_file, params['share_vocab'])
char_vocab_table = vocab_utils.get_char_table(enc_char_map_path)


src_data_file = 'D:/TDE/CharEncoder-WordDecoderS2S/data//srcData.txt'
tgt_data_file = 'D:/TDE/CharEncoder-WordDecoderS2S/data//tgtData.txt'
src_dataset = tf.data.TextLineDataset(src_data_file)
tgt_dataset = tf.data.TextLineDataset(tgt_data_file)

batched_iter = iterator_utils.get_iterator(src_dataset,
                                   tgt_dataset,
                                   char_vocab_table,
                                   src_vocab_table,
                                   tgt_vocab_table,
                                   batch_size=128,
                                   sos='<s>',
                                   eos='</s>',
                                   char_pad = '<unk>',
                                   num_buckets=1,
                                   num_epochs = 5,
                                   src_max_len=params['src_max_len'],
                                   tgt_max_len=params['tgt_max_len'],
                                   src_char_max_len = params['char_max_len']
                                   )

In [4]:
src_vocab_table

In [5]:
config_proto = utils.get_config_proto(log_device_placement=params['log_device_placement'])
sess = tf.Session(config=config_proto)
with sess.as_default():
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    sess.run(batched_iter.initializer)

In [25]:
elm =  batched_iter.get_next()
with sess.as_default():
    for i in range(1):
        k = sess.run(elm)
        print ( k[1].shape, k[2].shape)

(128, 14) (128, 16)


In [26]:
k[1][0]

array([    1,  1585,   386, 56891,  6850,   146,     2,     2,     2,
           2,     2,     2,     2,     2])

In [27]:
k[2][0]

array([  5, 283,   5,   5,   5,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2])

In [24]:
k[5][0]

5

In [29]:
pwd()

'C:\\Users\\psopori\\Desktop\\TDE\\CharEncoder-WordDecoderS2S\\data'

In [30]:
# Save val data
with open('D://TDE/CharEncoder-WordDecoderS2S/data/srcDevData.txt', mode='r', encoding='utf-8') as myfile:
    content1 = myfile.readlines()
        
with open('D://TDE/CharEncoder-WordDecoderS2S/data/tgtDevData.txt', mode='r', encoding='utf-8') as myfile:
    content2 = myfile.readlines()

In [42]:
for i in range(len(content1)):
    if(len(content1[i].strip().split(' ')) != len(content2[i].strip().split(' '))):
       print (i)

45
46
59
86
87
102
137
140
148
232
235
266
269
296
301
319
399
407
418
439
449
524
538
543
569
589
615
620
636
645
661
697
714
741
742
746
749
798
859
866
868
871
883
887
890
940
955
1024
1054
1055
1068
1074
1125
1159
1205
1207
1269
1274
1283
1299
1330
1399
1408
1410
1415
1433
1449
1452
1490
1518
1571
1613
1614
1651
1653
1666
1712
1714
1751
1761
1779
1804
1830
1836
1865
1889
1932
1949
1970
1985
2001
2051
2063
2083
2095
2099
2146
2148
2152
2155
2289
2290
2343
2391
2416
2442
2449
2502
2511
2521
2558
2571
2586
2599
2618
2625
2643
2651
2700
2765
2797
2826
2844
2850
2855
2858
2868
2889
2904
2916
3000
3004
3014
3019
3055
3061
3108
3120
3170
3191
3208
3209
3263
3264
3268
3272
3279
3292
3320
3346
3361
3388
3409
3421
3455
3469
3494
3520
3551
3606
3607
3613
3618
3667
3673
3694
3721
3737
3787
3790
3800
3801
3830
3854
3878
3879
3893
3903
3916
3935
3948
3952
3957
3979
3987
3998
4006
4065
4080
4108
4121
4147
4155
4180
4199
4200
4216
4220
4221
4231
4249
4253
4280
4308
4334
4365
4382
4384
4390
4422
44

59211
59288
59357
59365
59433
59451
59477
59508
59509
59561
59571
59585
59589
59599
59605
59626
59636
59652
59655
59656
59662
59665
59684
59722
59727
59773
59781
59842
59882
59898
59907
59908
59937
59949
59981
59983
60012
60024
60066
60132
60161
60179
60212
60220
60223
60234
60248
60262
60308
60323
60346
60398
60450
60471
60472
60522
60523
60525
60608
60654
60661
60734
60744
60754
60766
60771
60831
60916
60924
60928
61006
61059
61061
61066
61074
61168
61174
61201
61229
61260
61276
61281
61322
61371
61375
61469
61487
61493
61558
61560
61573
61584
61601
61606
61609
61618
61652
61696
61736
61752
61760
61762
61793
61796
61814
61817
61948
61983
61989
61997
62012
62016
62044
62090
62122
62188
62197
62224
62251
62272
62308
62316
62317
62319
62376
62395
62419
62420
62429
62475
62476
62501
62524
62570
62581
62636
62706
62715
62730
62778
62795
62811
62815
62829
62831
62836
62874
62891
62903
62905
62945
62954
63043
63051
63064
63120
63137
63153
63166
63197
63201
63214
63287
63300
63308
63310
6332

In [47]:
(content1[148].strip().split(' '))

['macys',
 'web',
 'pymt',
 'des:online',
 'pmt',
 'id:21407',
 'indn:39b6ccf911545aa4',
 'co',
 'id:1460']

In [48]:
(content2[148].strip().split(' '))

['1240',
 '164252',
 '164252',
 '164252',
 '164252',
 '164252',
 '164252',
 '164252',
 '164252',
 '164252',
 '164252',
 '164252',
 '164252']